In [1]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
from pathlib import Path
import pandas as pd
cwd = Path.cwd()
data_path = cwd / 'data'


from urllib.parse import quote_plus
import sqlalchemy

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};SERVER=server-dw01.database.windows.net;DATABASE=DataWarehouseStaging;UID=dw_userlogin;PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [13]:
def mid_dict_to_dwh(table_name, data_path):
    file_list = [f.parts[-1].replace('.csv','') for f in data_path.iterdir() if '01-' in f.parts[-1]]
    df = pd.DataFrame(data = [], columns = ['company_id', 'com_id_h'])
    for file_name in file_list:
            file_df = pd.read_csv(data_path / f'{file_name}.csv')
            if 'company_id' in file_df.columns:
                df = df.append(file_df[['company_id', 'com_id_h']])
    df = df.drop_duplicates()
    to_sql_fast(df, name = table_name, engine = ENGINE, if_exists = 'replace')

In [14]:
mid_dict_to_dwh('pam.MID_dict', data_path)